In [1]:
import pandas as pd

In [2]:
!ls datasets

dim_campaigns.csv dim_products.csv  dim_stores.csv    fact_events.csv


In [35]:
campaigns_df = pd.read_csv("datasets/dim_campaigns.csv")
products_df = pd.read_csv("datasets/dim_products.csv")
stores_df = pd.read_csv("datasets/dim_stores.csv")
events_df = pd.read_csv("datasets/fact_events.csv")


In [36]:
campaigns_df.head()

,campaign_id,campaign_name,start_date,end_date
0,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023
1,CAMP_SAN_01,Sankranti,10-01-2024,16-01-2024


In [ ]:
products_df.head()

In [38]:
stores_df.head()

,store_id,city
0,STTRV-0,Trivandrum
1,STMDU-3,Madurai
2,STHYD-6,Hyderabad
3,STVSK-1,Visakhapatnam
4,STCBE-3,Coimbatore


In [39]:
events_df.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272


In [40]:
events_df.shape

(1510, 9)

In [41]:
events_df.duplicated(subset=['store_id', 'product_code', 'campaign_id']).sum()

np.int64(10)

In [42]:
events_df.drop_duplicates(subset=['store_id', 'product_code', 'campaign_id'], inplace=True)

In [43]:
events_df.shape

(1500, 9)

## 10 rows were dropped

In [60]:
stores_per_city = stores_df.groupby("city")["store_id"].nunique()

In [68]:
stores_per_city[stores_per_city>5]

city
Bengaluru    10
Chennai       8
Hyderabad     7
Name: store_id, dtype: int64

## 3 cities have more than 5 stores

In [69]:
events_df.isnull().sum()

event_id                        0
store_id                        0
campaign_id                     0
product_code                    0
base_price(before_promo)        0
quantity_sold(before_promo)    20
promo_type                      0
base_price(after_promo)         0
quantity_sold(after_promo)      0
dtype: int64

In [75]:
med = events_df['quantity_sold(before_promo)'].median()
med 

78.0

In [78]:
events_df['quantity_sold(before_promo)'] = events_df['quantity_sold(before_promo)'].fillna(value=med)

In [79]:
events_df.isnull().sum()

event_id                       0
store_id                       0
campaign_id                    0
product_code                   0
base_price(before_promo)       0
quantity_sold(before_promo)    0
promo_type                     0
base_price(after_promo)        0
quantity_sold(after_promo)     0
dtype: int64

## 20 null values and median is 78

In [97]:
event_product = events_df.merge(products_df,on="product_code", how="left")

In [100]:
cat_min = event_product.groupby('category')['base_price(before_promo)'].min()
cat_min

category
Combo1               3000
Grocery & Staples     156
Home Appliances       350
Home Care              55
Personal Care          50
Name: base_price(before_promo), dtype: int64

In [101]:
cat_min.idxmin()

'Personal Care'

In [102]:
cat_min.min()

50

## min base price before promo is 50 and product catergory is `Personal care`

In [103]:
event_campaign = events_df.merge(campaigns_df, on="campaign_id", how="left")
event_campaign.sample(1)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date
1281,1e8961,STBLR-3,CAMP_SAN_01,P12,62,61.0,50% OFF,31,84,Sankranti,10-01-2024,16-01-2024


In [108]:
event_campaign.loc[(event_campaign["promo_type"] == "BOGOF") &  (event_campaign["campaign_name"] == "Diwali"), "quantity_sold(after_promo)"].sum()

np.int64(34461)

## 34461 total quantity sold after the promotion for the BOGOF promo type during the Diwali campaign

In [121]:
event_campaign[event_campaign["campaign_name"] == "Diwali"]["quantity_sold(after_promo)"].max()

1545

In [128]:
sales_by_campaign_and_store = event_campaign.groupby(["campaign_name", "store_id"])["quantity_sold(after_promo)"].sum()
sales_by_campaign_and_store

campaign_name  store_id
Diwali         STBLR-0     4759
               STBLR-1     4146
               STBLR-2     4169
               STBLR-3     4373
               STBLR-4     4408
                           ... 
Sankranti      STVSK-0     4522
               STVSK-1     4536
               STVSK-2     4394
               STVSK-3     2892
               STVSK-4     3065
Name: quantity_sold(after_promo), Length: 100, dtype: int64

In [131]:
sales_by_campaign_and_store["Diwali"].idxmax()

'STCHE-4'

## STCHE-4 had maximum diwali sales

In [132]:
sales_by_campaign_and_store["Diwali"].max()

5013

In [115]:
sales_after_campaign = event_campaign.groupby("campaign_name")[["quantity_sold(before_promo)","quantity_sold(after_promo)"]].sum()
sales_after_campaign

,quantity_sold(before_promo),quantity_sold(after_promo)
campaign_name,,
Diwali,109756.0,183404
Sankranti,97894.0,252069


In [133]:
sales_after_campaign["change"] = (sales_after_campaign["quantity_sold(after_promo)"] - sales_after_campaign["quantity_sold(before_promo)"])

In [134]:
sales_after_campaign

,quantity_sold(before_promo),quantity_sold(after_promo),change
campaign_name,,,
Diwali,109756.0,183404,73648.0
Sankranti,97894.0,252069,154175.0


In [141]:
event_campaign.head(2)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024


In [142]:
event_campaign["before_promo_revenue"] = event_campaign["base_price(before_promo)"]*event_campaign["quantity_sold(before_promo)"]
event_campaign["after_promo_revenue"] = event_campaign["base_price(after_promo)"]*event_campaign["quantity_sold(after_promo)"]

In [143]:
event_campaign.head(2)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,before_promo_revenue,after_promo_revenue
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,94830.0,62279
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,140230.0,300070


In [156]:
event_campaign["revenue_increment"] = event_campaign["after_promo_revenue"] - event_campaign["before_promo_revenue"]

In [157]:
event_campaign["IR%"] = event_campaign["revenue_increment"] * 100/ event_campaign["before_promo_revenue"]

In [158]:
event_campaign.head(2)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,before_promo_revenue,after_promo_revenue,IR%,revenue_increment
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,94830.0,62279,-34.325635,-32551.0
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,140230.0,300070,113.984169,159840.0


In [177]:
product_code_revenue = event_campaign.groupby(["campaign_name","product_code"], as_index=False)[["before_promo_revenue","after_promo_revenue"]].sum()

In [183]:
sankranti = (product_code_revenue[product_code_revenue["campaign_name"] == "Sankranti"]).copy()
sankranti

,campaign_name,product_code,before_promo_revenue,after_promo_revenue
15,Sankranti,P01,2108376.0,2011235
16,Sankranti,P02,18374760.0,17262720
17,Sankranti,P03,3189600.0,6118500
18,Sankranti,P04,6813550.0,12779800
19,Sankranti,P05,59675.0,35875
20,Sankranti,P06,561080.0,342722
21,Sankranti,P07,583500.0,1065450
22,Sankranti,P08,2485910.0,4649330
23,Sankranti,P09,239040.0,146462
24,Sankranti,P10,75400.0,42328


In [184]:
sankranti["revenue_change"] = sankranti.after_promo_revenue - sankranti.before_promo_revenue

In [185]:
sankranti["IR%"] = sankranti.revenue_change*100/sankranti.before_promo_revenue

In [187]:
sankranti["IR%"].idxmax()

np.int64(17)

In [189]:
sankranti.loc[17]

campaign_name           Sankranti
product_code                  P03
before_promo_revenue    3189600.0
after_promo_revenue       6118500
revenue_change          2928900.0
IR%                     91.826561
Name: 17, dtype: object

In [190]:
sankranti

,campaign_name,product_code,before_promo_revenue,after_promo_revenue,revenue_change,IR%
15,Sankranti,P01,2108376.0,2011235,-97141.0,-4.607385
16,Sankranti,P02,18374760.0,17262720,-1112040.0,-6.051997
17,Sankranti,P03,3189600.0,6118500,2928900.0,91.826561
18,Sankranti,P04,6813550.0,12779800,5966250.0,87.564485
19,Sankranti,P05,59675.0,35875,-23800.0,-39.882698
20,Sankranti,P06,561080.0,342722,-218358.0,-38.917445
21,Sankranti,P07,583500.0,1065450,481950.0,82.596401
22,Sankranti,P08,2485910.0,4649330,2163420.0,87.027286
23,Sankranti,P09,239040.0,146462,-92578.0,-38.729083
24,Sankranti,P10,75400.0,42328,-33072.0,-43.862069


In [191]:
event_campaign.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,before_promo_revenue,after_promo_revenue,IR%,revenue_increment
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,94830.0,62279,-34.325635,-32551.0
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,140230.0,300070,113.984169,159840.0
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Diwali,12-11-2023,18-11-2023,289820.0,281088,-3.012905,-8732.0
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Diwali,12-11-2023,18-11-2023,6710.0,4387,-34.619970,-2323.0
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Sankranti,10-01-2024,16-01-2024,366000.0,680000,85.792350,314000.0


In [192]:
event_camp_store = events_df.merge(campaigns_df, on="campaign_id", how="left").merge(stores_df, on="store_id", how="left" )

In [193]:
event_camp_store

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,city
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Chennai
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Chennai
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Diwali,12-11-2023,18-11-2023,Bengaluru
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Diwali,12-11-2023,18-11-2023,Bengaluru
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Sankranti,10-01-2024,16-01-2024,Hyderabad
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,9bd616,STCHE-0,CAMP_DIW_01,P01,172,330.0,33% OFF,115,504,Diwali,12-11-2023,18-11-2023,Chennai
1496,fe4a15,STCBE-4,CAMP_SAN_01,P01,172,205.0,33% OFF,115,284,Sankranti,10-01-2024,16-01-2024,Coimbatore
1497,2f0b42,STVSK-1,CAMP_SAN_01,P01,172,78.0,33% OFF,115,302,Sankranti,10-01-2024,16-01-2024,Visakhapatnam
1498,339bf7,STCHE-5,CAMP_DIW_01,P05,55,127.0,25% OFF,41,109,Diwali,12-11-2023,18-11-2023,Chennai


In [203]:
df_9 = event_camp_store[(event_camp_store["city"] == "Visakhapatnam") &  (event_camp_store["campaign_name"] == "Diwali")]

In [206]:
isu_9 = df_9.groupby("store_id")[["quantity_sold(after_promo)","quantity_sold(before_promo)"]].sum()

In [207]:
isu_9

,quantity_sold(after_promo),quantity_sold(before_promo)
store_id,,
STVSK-0,3005,1768.0
STVSK-1,3078,1903.0
STVSK-2,2860,1701.0
STVSK-3,2656,1780.0
STVSK-4,2908,1926.0


In [210]:
isu_9["ISU%"] = (isu_9["quantity_sold(after_promo)"] - isu_9["quantity_sold(before_promo)"])*100/isu_9["quantity_sold(before_promo)"]

In [211]:
isu_9

,quantity_sold(after_promo),quantity_sold(before_promo),ISU%
store_id,,,
STVSK-0,3005,1768.0,69.966063
STVSK-1,3078,1903.0,61.744614
STVSK-2,2860,1701.0,68.136390
STVSK-3,2656,1780.0,49.213483
STVSK-4,2908,1926.0,50.986501


In [212]:
san = event_camp_store[(event_camp_store["campaign_name"] == "Sankranti")]

In [215]:
san.sample(1)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,city
535,4e6a19,STHYD-3,CAMP_SAN_01,P07,300,45.0,BOGOF,150,121,Sankranti,10-01-2024,16-01-2024,Hyderabad


In [220]:
san_promo = san.groupby("promo_type")[["quantity_sold(before_promo)","base_price(before_promo)","quantity_sold(after_promo)","base_price(after_promo)"]].sum()

In [221]:
san_promo

,quantity_sold(before_promo),base_price(before_promo),quantity_sold(after_promo),base_price(after_promo)
promo_type,,,,
25% OFF,6601.0,30500,5307,22800
33% OFF,33624.0,51600,47459,34550
50% OFF,4451.0,12600,6100,6300
500 Cashback,5395.0,150000,12411,125000
BOGOF,47823.0,171500,180792,85750


In [224]:
san_promo["rev_before"] = san_promo["quantity_sold(before_promo)"]*san_promo["base_price(before_promo)"]
san_promo["rev_after"] = san_promo["quantity_sold(after_promo)"]*san_promo["base_price(after_promo)"]
san_promo

,quantity_sold(before_promo),base_price(before_promo),quantity_sold(after_promo),base_price(after_promo),rev_before,rev_after
promo_type,,,,,,
25% OFF,6601.0,30500,5307,22800,2.013305e+08,120999600
33% OFF,33624.0,51600,47459,34550,1.734998e+09,1639708450
50% OFF,4451.0,12600,6100,6300,5.608260e+07,38430000
500 Cashback,5395.0,150000,12411,125000,8.092500e+08,1551375000
BOGOF,47823.0,171500,180792,85750,8.201644e+09,15502914000


In [226]:
san_promo["ISU%"] = (san_promo["quantity_sold(after_promo)"] - san_promo["quantity_sold(before_promo)"])*100/san_promo["quantity_sold(before_promo)"]
san_promo["IR%"] = (san_promo["rev_after"] - san_promo["rev_before"])*100/san_promo["rev_before"]

In [227]:
san_promo[["ISU%", "IR%"]]

,ISU%,IR%
promo_type,,
25% OFF,-19.603090,-39.900015
33% OFF,41.146205,-5.492221
50% OFF,37.047854,-31.476073
500 Cashback,130.046339,91.705283
BOGOF,278.044037,89.022019


In [233]:
sankranti = event_camp_store[(event_camp_store["campaign_name"] == "Sankranti")].copy()
sankranti["revenue_before"] = sankranti["base_price(before_promo)"] * sankranti["quantity_sold(before_promo)"]
sankranti["revenue_after"]  = sankranti["base_price(after_promo)"] * sankranti["quantity_sold(after_promo)"]


In [234]:
promo_stats = sankranti.groupby("promo_type", as_index=False).agg({
    "quantity_sold(before_promo)": "sum",
    "quantity_sold(after_promo)": "sum",
    "revenue_before": "sum",
    "revenue_after": "sum"
})

In [235]:
# Calculate ISU% and IR%
promo_stats["ISU%"] = ((promo_stats["quantity_sold(after_promo)"] - promo_stats["quantity_sold(before_promo)"]) /
                        promo_stats["quantity_sold(before_promo)"]) * 100

promo_stats["IR%"] = ((promo_stats["revenue_after"] - promo_stats["revenue_before"]) /
                       promo_stats["revenue_before"]) * 100

In [236]:
promo_stats

,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),revenue_before,revenue_after,ISU%,IR%
0,25% OFF,6601.0,5307,935195.0,567387,-19.603090,-39.329552
1,33% OFF,33624.0,47459,20483136.0,19273955,41.146205,-5.903300
2,50% OFF,4451.0,6100,527482.0,363564,37.047854,-31.075563
3,500 Cashback,5395.0,12411,16185000.0,31027500,130.046339,91.705283
4,BOGOF,47823.0,180792,19355170.0,36451055,278.044037,88.327227


In [240]:
san_promo

,quantity_sold(before_promo),base_price(before_promo),quantity_sold(after_promo),base_price(after_promo),rev_before,rev_after,ISU%,IR%
promo_type,,,,,,,,
25% OFF,6601.0,30500,5307,22800,2.013305e+08,120999600,-19.603090,-39.900015
33% OFF,33624.0,51600,47459,34550,1.734998e+09,1639708450,41.146205,-5.492221
50% OFF,4451.0,12600,6100,6300,5.608260e+07,38430000,37.047854,-31.476073
500 Cashback,5395.0,150000,12411,125000,8.092500e+08,1551375000,130.046339,91.705283
BOGOF,47823.0,171500,180792,85750,8.201644e+09,15502914000,278.044037,89.022019


In [238]:
negative_promos = promo_stats[(promo_stats["ISU%"] < 0) & (promo_stats["IR%"] < 0)]


In [239]:
print("Promo types with negative IR% and ISU% during Sankranti campaign:")
print(negative_promos[["promo_type", "ISU%", "IR%"]])

Promo types with negative IR% and ISU% during Sankranti campaign:
  promo_type      ISU%        IR%
0    25% OFF -19.60309 -39.329552
